In [ ]:
import os
from groq import Groq
import re
from typing import List, Dict, Any
import yaml
from tqdm import tqdm
import json

In [ ]:
API_KEY = ""
DJANGO_PROJECT_PATH = "/Users/ryanmarr/Documents/saleor"
MODEL_NAME = "meta-llama/llama-4-scout-17b-16e-instruct"
MAX_TOKENS = 2048
TEMPERATURE = 0.1
TOP_P = 1
STREAM = False

In [ ]:
# Groq client setup
client = Groq(api_key=API_KEY)

In [ ]:
def find_django_files(directory: str) -> List[str]:
    """Perform DFS to find all Python files in Django project"""
    django_files = []
    
    for root, dirs, files in os.walk(directory):
        # Skip common directories that don't contain Django code
        dirs[:] = [d for d in dirs if not d.startswith('.') and d not in ['node_modules', '__pycache__', 'venv', 'env', '.git']]
        
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                django_files.append(file_path)
    
    return django_files

In [ ]:
def extract_rest_apis_from_file_with_groq(file_path: str) -> List[Dict[str, Any]]:
    """Use Groq to analyze file content and find REST APIs"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()  
        
        # Skip files that don't contain common Django/API keywords
        if not any(keyword in content.lower() for keyword in ['api', 'view', 'rest', 'http', 'request', 'response', 'serializer']):
            return []
        
        prompt = f"""
        Analyze this Python file and identify all Django REST API functions or classes.
        
        File: {file_path}
        
        Look for:
        1. Functions that handle HTTP methods (GET, POST, PUT, DELETE, PATCH)
        2. Functions that process requests and return responses
        3. Any other REST API endpoints
        
        File content:
        {content}  # First 4000 chars for analysis
        
        IMPORTANT: Return ONLY valid JSON with this exact structure:
        {{
            "apis": [
                {{
                    "name": "function_or_class_name",
                    "http_method": "GET|POST|PUT|DELETE|PATCH|UNKNOWN",
                    "description": "Brief description of what this API does",
                    "content_django": actual function and implementation,
                    "content_dafny": Based on the django function, create a Dafny function specification with preconditions and postconditions
                }}
            ]
        }}
        
        If no REST APIs are found, return: {{"apis": []}}
        Do not include any text before or after the JSON.
        """
        
        try:
            completion = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                temperature=TEMPERATURE,  # Lower temperature for more consistent JSON
                max_tokens=MAX_TOKENS,
                top_p=TOP_P,
                stream=STREAM
            )
            
            response = completion.choices[0].message.content.strip()
            # Try to extract JSON from the response
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                try:
                    result = json.loads(json_str)
                    return result
                    # print(f'result: {result}')
                    # apis = []
                    
                    # for api_info in result.get('apis', []):
                    #     apis.append({
                    #         'name': api_info.get('name', 'unknown'),
                    #         'file': file_path,
                    #         'content': content,
                    #         'line': api_info.get('line_number', 1),
                    #         'type': api_info.get('type', 'function'),
                    #         'http_method': api_info.get('http_method', 'UNKNOWN'),
                    #         'description': api_info.get('description', '')
                    #     })
                    
                    # return apis
                    
                except json.JSONDecodeError:
                    print(f"Failed to parse JSON from response for {file_path}")
                    print(f"Response: {response}")
                    print(f"json_match: {json_match}")
                    return []
            else:
                print(f"No JSON found in response for {file_path}")
                print(f"Response: {response}")
                return []
                
        except Exception as e:
            print(f"Error calling Groq API for {file_path}: {e}")
            return []
            
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return [] 

In [ ]:
# Main execution function for notebook
"""Analyze Django project and generate Dafny specifications"""

print(f"Analyzing Django project at: {django_project_path}")

# Find all Python files
django_files = find_django_files(django_project_path)
print(f"Found {len(django_files)} Python files")

# Extract REST APIs
all_apis = []
for file_path in tqdm(django_files, desc="Analyzing files"):
    apis = extract_rest_apis_from_file_with_groq(file_path)
    all_apis.extend(apis)